In [1]:
from typing import Annotated, TypedDict, List, Dict, Any, Optional
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from IPython.display import Image, display
import gradio as gr
import uuid
from dotenv import load_dotenv


d:\Jeevan\Agentic_AI\LangGraph_Langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class EvaluatorOutput(BaseModel):
    feedback: str = Field(description="Feedback on the workers response")
    success_criteria_met: bool = Field(description= "Whether the success criteria has benn met")
    user_input_needed: bool = Field(description="True if more input is needed from the user, or clarifications, or the assistant is stuck")

In [3]:
# The state

class State(TypedDict):
    messages: Annotated[List[Any], add_messages]
    success_criteria: str
    feedback_on_work: Optional[str]
    success_criteria_met: bool
    user_input_needed: bool


In [4]:
# Initialize the LLM

worker_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
worker_llm_with_tools = worker_llm.bind_tools(tools)


evaluator_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
evaluator_llm_with_structured_output = evaluator_llm.with_structured_output(EvaluatorOutput)


NameError: name 'tools' is not defined

In [12]:
# The worker node

def worker(state: State) -> Dict[str, Any]:
    system_message= f"""you are a helpful assistant that can use tools to complete tasks
    You keep working on a task until either you have a question or clarification for the user, or the success criteria is met
    This is the success criteria: {state["success_criteria"]}
    you should reply either with a question for the user about this assignment, or with your final response.
    If you have a question for the user, you need to reply by clearly stating that you are asking a question, and then ask the question.
    
    An example might be:

    Question: Please clarify whether you want a summary or a detailed answer

    If you have finished, reply with the final answer, and don't ask a question. simply reply with the answer.
"""

    if state.get("feedback_on_work"):
        system_message += f"""
        Previously you thought you completed the assignment, but your reply was rejected because the success criteria was not met.
        Here is the feedback of why this was rejected: {state["feedback_on_work"]}
        With this feedback in mind, please continue the assignment, ensuring that you meet the success criteria or ask for more information if needed.
        """

        # Add in the system message to the messages

        found_system_message = False
        messages = state["messages"]
        for message in messages:
            if isinstance(message, SystemMessage):
                message.content = system_message
                found_system_message = True
        
        if not found_system_message:
            messages = [SystemMessage(content=system_message)] + messages

        # Invoke the LLM with tools
        response = [SystemMessage(content=system_message)] + messages

        # return the updated state
        return {"messages": [response],
        }

In [5]:
def worker_router(state: State) -> str:
    last_message = state["messages"][-1]
    
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    else:
        return "evaluator"
        

In [6]:
def format_conversation(messages: List[Any]) -> str:
    conversation = "Conversation history:\n\n"
    for message in messages:
        if isinstance(message, HumanMessage):
            conversation += f"User: {message.content}\n"
        elif isinstance(message, AIMessage):
            text = message.content or "[Tools use]"
            conversation += f"Assistant: {text}\n"
    return conversation

In [8]:
def evaluator(state: State) -> State:
    last_response = state["messages"][-1].content

    system_message = f"""You are an evaluator that determines whether the assistant has met the success criteria.
    Assess the Assistant's last response based on the given criteria. Respond with your feedback, and with your decision on whether the success criteria has been met,
    and whether the user needs to provide more information."""

    user_message = f"""You are evaluating a conversation between the user and assistant. you decide what action to take based on the last response
    The entire conversation with assistant, with the user's original request and all replies is:
    {format_conversation(state['messages'])}

    The success criteria for this assignment is:
    {state['success_criteria']}

    And the final response from the Assistant that you are evaluating is:
    {last_response}

    Respond with your feedback, and with your decision on whether the success criteria has been met,
    Also decide if more user input is required, either because the assistant is stuck, or because the user needs to clarify their request."""
    
    if state["feedback_on_work"]:
        user_message += f"Also, note that in a prior atttempt from the assistant, you provided this feedback: {state['feedback_on_work']}"
        user_message += "If you're seeing the Assistant repeating the same mistakes, then consider responding that user input is needed."

    evaluator_messages = [SystemMessage(content=system_message), HumanMessage(content=user_message)]
    
    eval_result = evaluator_llm_with_output.invoke(evaluator_messages)
    new_state = {
        "messages": [{"role":"assistant", "content": f"Evaluator Feedback on this answer: {eval_result.feedback}"}],
        "feedback_on_work": eval_result.feedback,
        "success_criteria_met": eval_result.success_criteria_met,
        "user_input_needed": eval_result.user_input_needed,
    }
    return new_state

In [10]:
def route_based_on_evaluator_result(state: State) -> str:
    if state["success_criteria_met"] or state["user_input_needed"]:
        return "END"
    else:
        return "worker"


In [14]:
# set up graph builder with state

graph_builder = StateGraph(State)

# Add nodes
graph_builder.add_node("worker", worker)
graph_builder.add_node("tools", ToolNode(tools=tools))
graph_builder.add_node("evaluator", evaluator)

# Add edges
graph_builder.add_edge(START, "worker")
graph_builder.add_conditional_edges("worker", worker_router, {"tools": "tools", "evaluator": "evaluator"})
graph_builder.add_edge("tools", "worker")
graph_builder.add_conditional_edges("evaluator", route_based_on_evaluator_result, {"END": END, "worker": "worker"})

# compile the Graph
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

NameError: name 'tools' is not defined

In [15]:
display(Image(graph.get_graph().draw_mermaid_png()))

NameError: name 'graph' is not defined

Next comes the gradio Callback to kick-off a super step